http://aicompanion.or.kr/kor/tech/data.php 자율지능 디지털동반자 사이트(이전 ai-hub)

In [1]:
import pandas as pd
import plotly.express as px

In [2]:
df1 = pd.read_csv("./감성대화/감성대화말뭉치(최종데이터)_Training.csv", encoding='UTF-8')
df2 = pd.read_csv("./감성대화/감성대화말뭉치(최종데이터)_Validation.csv", encoding='UTF-8')

In [3]:
df1 = df1[['감정_대분류', '사람문장1']] # 감성대화중 감정 대분류에 해당하는 것은 '사람문장1'이기 때문에 '감정대분류'와 '사람문장1'만 남겨놓음.
df1.rename(columns={"감정_대분류": "sentiment", "사람문장1":"sentence"}, inplace=True) # column명 통일
df1 = df1[["sentence", "sentiment"]] # column순서 변경

# df2에 대해서도 동일한 과정 실시
df2 = df2[['감정_대분류', '사람문장1']]
df2.rename(columns={"감정_대분류": "sentiment", "사람문장1":"sentence"}, inplace=True)
df2 = df2[["sentence", "sentiment"]]

In [4]:
result = pd.concat([df1,df2])
result.drop_duplicates(inplace=True)
result.reset_index(drop=True, inplace=True)
result

,sentence,sentiment
0,일은 왜 해도 해도 끝이 없을까? 화가 난다.,분노
1,이번 달에 또 급여가 깎였어! 물가는 오르는데 월급만 자꾸 깎이니까 너무 화가 나.,분노
2,회사에 신입이 들어왔는데 말투가 거슬려. 그런 애를 매일 봐야 한다고 생각하니까 스...,분노
3,직장에서 막내라는 이유로 나에게만 온갖 심부름을 시켜. 일도 많은 데 정말 분하고 ...,분노
4,얼마 전 입사한 신입사원이 나를 무시하는 것 같아서 너무 화가 나.,분노
...,...,...
58251,나랑 비슷한 시기에 결혼하는 친구는 시댁에서 집을 해줘서 너무 부러워. 우리는 대출...,당황
58252,친구 한 명이 결혼해서 아이를 가졌는데 너무 행복해 보이더라. 기분이 좋지만은 않아.,당황
58253,남들은 결혼 전에 일억을 모았다는데 난 뭐를 한 것인지 모르겠어. 자괴감만 드네.,당황
58254,나보다 결혼을 먼저 한 친구가 부러워. 그 친구 남편은 직장도 내 남편보다 좋고 키...,당황


In [5]:
result.isnull().sum()

sentence     0
sentiment    0
dtype: int64

In [6]:
result['sentiment'].unique()

array(['분노', '기쁨', '불안', '당황', '슬픔', '상처'], dtype=object)

In [7]:
# "sentiment" 컬럼의 값들의 분포를 막대그래프로 그리기
fig = px.histogram(result, x='sentiment', title='Sentiment Distribution')

# fig의 크기 조절 (width와 height를 원하는 크기로 설정합니다.)
fig.update_layout(
    width=800,  # 가로 크기
    height=500,  # 세로 크기
    xaxis=dict(tickfont=dict(size=14, family='Arial, bold')),  # x축 레이블 폰트 크기와 스타일 조절
    yaxis=dict(tickfont=dict(size=14, family='Arial, bold')),  # y축 레이블 폰트 크기와 스타일 조절
)

# 색상을 랜덤으로 지정
# 'color_discrete_sequence' 파라미터에 'px.colors.qualitative'를 사용하면 Plotly에서 제공하는 랜덤 색상을 사용할 수 있습니다.
fig.update_traces(marker=dict(color=px.colors.qualitative.Prism))

# 막대그래프의 모양 변경 (width 속성을 여기서 조절)
fig.update_traces(
    marker=dict(color=px.colors.qualitative.Prism),  # 색상을 랜덤하게 지정
    marker_line_width=3,  # 테두리 선 두께 조절
    marker_line_color='black',  # 테두리 선 색상 조절
    opacity=0.9,  # 막대의 투명도 조절 (0부터 1 사이의 값을 사용, 1은 완전 불투명)
)

# 그래프 출력
fig.show()

In [8]:
result.to_csv("./dataset/train_data.csv", index=False, encoding='UTF-8')

In [9]:
sentiment_distribution = result['sentiment'].value_counts()

print(sentiment_distribution)

불안    10432
분노    10417
상처    10145
슬픔    10125
당황     9799
기쁨     7338
Name: sentiment, dtype: int64


In [10]:
df3 = pd.read_csv("./감정 분류를 위한 대화 음성 데이터셋/4차년도.csv", encoding='cp949')
df3.rename(columns={'4번감정세기':'4번 감정세기'}, inplace=True)

df4 = pd.read_csv("./감정 분류를 위한 대화 음성 데이터셋/5차년도.csv", encoding='cp949')
df4.rename(columns={'4번감정세기':'4번 감정세기'}, inplace=True)

df5 = pd.read_csv("./감정 분류를 위한 대화 음성 데이터셋/5차년도_2차.csv", encoding='cp949')
df5.rename(columns={'4번감정세기':'4번 감정세기'}, inplace=True)

In [11]:
def emo_sum(dataframe): # 각 row의 감정세기를 계산하는 함수
    rep_emo = []
    for index, row in dataframe.iterrows():
        emo = {}
        for i in dataframe.columns[3:13:2]:
            emo[row[i]] = emo.get(row[i], 0) + row[i+"세기"]
        # 딕셔너리의 값들 중에서 최댓값 찾기
        max_value = max(emo.values())

        # 최댓값에 해당하는 key 찾기
        max_key_list = [key for key, value in emo.items() if value == max_value]
        if len(max_key_list) > 1:
            rep_emo.append(None)
        else:
            rep_emo.append(max_key_list[0])
    return rep_emo

In [12]:
df3["sentiment"] = emo_sum(df3) # 함수를 통해 출력된 감정 리스트를 데이터프레임에 추가
df3 = df3[['발화문', 'sentiment']] # 문장과 감정만 남겨놓음
df3.dropna(inplace=True)
df3.rename(columns={'발화문': 'sentence'}, inplace=True) # column명 통일

df4["sentiment"] = emo_sum(df4) # 함수를 통해 출력된 감정 리스트를 데이터프레임에 추가
df4 = df4[['발화문', 'sentiment']] # 문장과 감정만 남겨놓음
df4.dropna(inplace=True)
df4.rename(columns={'발화문': 'sentence'}, inplace=True) # column명 통일

df5["sentiment"] = emo_sum(df5) # 함수를 통해 출력된 감정 리스트를 데이터프레임에 추가
df5 = df5[['발화문', 'sentiment']] # 문장과 감정만 남겨놓음
df5.rename(columns={'발화문': 'sentence'}, inplace=True) # column명 통일
df5.dropna(inplace=True)
df5['sentiment'] = df5['sentiment'].apply(lambda x : x.capitalize())

In [13]:
result2 = pd.concat([df3,df4,df5])
result2.drop_duplicates(inplace=True, keep='last')
result2.reset_index(drop=True, inplace=True)
result2

,sentence,sentiment
0,"어, 청소 니가 대신 해 줘!",Angry
1,둘 다 청소 하기 싫어. 귀찮아.,Angry
2,둘 다 하기 싫어서 화내.,Angry
3,그럼 방세는 어떡해.,Sadness
4,권태긴줄 알았는데 다른 사람이 생겼나보더라고.,Sadness
...,...,...
33959,나 엘리베이터에 갇혔어.,Sadness
33960,하지만 기분이 나쁜 걸 어떡해?,Sadness
33961,자취방 엘리베이턴데 정전인가봐.,Sadness
33962,나 드디어 프로젝트 끝났어!,Sadness


In [42]:
result2[result2['sentiment'] == 'Fear']

,sentence,sentiment
150,어제 저녁에 진짜 무서웠어.,Fear
151,지진이 났잖아. 나는 괜찮았는데 내 친구는 발목을 다쳤어.,Fear
167,응 그래야겠어. 지진을 처음 겪어봐서 진짜 너무 무섭더라.,Fear
180,어제 갑자기 지진이 일어났거든.,Fear
221,"친구와 밥 먹는데, 심한 지진이 일어났어.",Fear
...,...,...
33862,어떤 사람들끼리 칼부림을 했나봐.,Fear
33876,여기 아파트 B동이야. 빨리 와줘.,Fear
33900,우리 이러다 죽는 거 아니겠지?,Fear
33939,혼자 있는데 너무 무서워.,Fear


In [61]:
result2[result2['sentiment'] == 'Sadness']

,sentence,sentiment
3,그럼 방세는 어떡해.,Sadness
4,권태긴줄 알았는데 다른 사람이 생겼나보더라고.,Sadness
7,내가 애정 표현을 잘 못해서 자주 싸우긴 했어.,Sadness
8,그러고 싶은데 보증금 때문에 그럴 수가 없어.,Sadness
9,이 회사가 이번 시즌 마지막 회사였어.,Sadness
...,...,...
33958,맞아. 나한테만 퉁명스럽고 일을 많이 시키더라고.,Sadness
33959,나 엘리베이터에 갇혔어.,Sadness
33960,하지만 기분이 나쁜 걸 어떡해?,Sadness
33961,자취방 엘리베이턴데 정전인가봐.,Sadness


In [14]:
result2[result2['sentiment'] == 'Surprise']

,sentence,sentiment
31,어제 지진 있었잖아. 다행히 다치진 않았어.,Surprise
46,어제 지진 난 거 알아?,Surprise
148,지금은 괜찮은데 어제 완전 놀랬어.,Surprise
194,으 다른 층 써야될 거 같아. 깜짝 놀랐네.,Surprise
312,그대로 있어서 놀랐어!,Surprise
...,...,...
33877,나 아까 산책시키다가 너무 놀랬어.,Surprise
33893,진짜 너가 다 돈 줄 거야?,Surprise
33903,나 오늘 짭새가 모여있길래 구경갔다왔어.,Surprise
33922,아까 해피 산책시키다가 엄청 놀랬어.,Surprise


In [36]:
result2.isnull().sum()

sentence     0
sentiment    0
dtype: int64

In [37]:
result2['sentiment'].unique()

array(['Angry', 'Sadness', 'Disgust', 'Happiness', 'Neutral', 'Surprise',
       'Fear'], dtype=object)

In [38]:
# "sentiment" 컬럼의 값들의 분포를 막대그래프로 그리기
fig = px.histogram(result2, x='sentiment', title='Sentiment Distribution')

# fig의 크기 조절 (width와 height를 원하는 크기로 설정합니다.)
fig.update_layout(
    width=800,  # 가로 크기
    height=500,  # 세로 크기
    xaxis=dict(tickfont=dict(size=14, family='Arial, bold')),  # x축 레이블 폰트 크기와 스타일 조절
    yaxis=dict(tickfont=dict(size=14, family='Arial, bold')),  # y축 레이블 폰트 크기와 스타일 조절
)

# 색상을 랜덤으로 지정
# 'color_discrete_sequence' 파라미터에 'px.colors.qualitative'를 사용하면 Plotly에서 제공하는 랜덤 색상을 사용할 수 있습니다.
fig.update_traces(marker=dict(color=px.colors.qualitative.Prism))

# 막대그래프의 모양 변경 (width 속성을 여기서 조절)
fig.update_traces(
    marker=dict(color=px.colors.qualitative.Prism),  # 색상을 랜덤하게 지정
    marker_line_width=3,  # 테두리 선 두께 조절
    marker_line_color='black',  # 테두리 선 색상 조절
    opacity=0.9,  # 막대의 투명도 조절 (0부터 1 사이의 값을 사용, 1은 완전 불투명)
)

# 그래프 출력
fig.show()

In [39]:
sentiment_distribution = result2['sentiment'].value_counts()

print(sentiment_distribution)

Sadness      16309
Angry         7263
Happiness     4016
Fear          2409
Disgust       2260
Surprise      1173
Neutral        534
Name: sentiment, dtype: int64
